In [ ]:
import s4
from s4 import clarity
import keyring
import xlrd
import os
from datetime import datetime
import pytz
from datetime import datetime
from s4.clarity.artifact import Artifact
from s4.clarity.configuration import Workflow
from s4.clarity.queue import Queue
from s4.clarity.routing import Router
from s4.clarity.configuration import Stage

# dev, staging, or prod
server='prod'

# account settings
username = 'bgriffiths'
password = keyring.get_password(f"clarity-{server}", username)

# lims object
lims = s4.clarity.LIMS(f"https://billiontoone-{server}.claritylims.com/api/v2/", username, password)
print(f'API version: {lims.versions[0]['major']}')

In [ ]:
artifact_ids = ["2-########"]
#workflow names
remove_from='SMA NIPT v1.0' 
move_to='SMA v2.5'

#step names
remove_from_stage='cfDNA Extraction - Automated'
move_to_stage='Verification'

# Where to save the reports (suggest google docs)
dir1=r"G:\My Drive\Clarity Work"

if not os.path.exists(dir1):
    raise Exception(f"Couldn't find path {dir1}")

print(f'Processing {len(artifact_ids)} samples')

In [ ]:
header="""
    <!DOCTYPE html>
                        <html lang="en">
                        <head>
                            <meta name="btolog">
                            <meta charset="UTF-8">
                            <meta name="viewport" content="width=device-width, initial-scale=1.0">
                            <meta http-equiv="X-UA-Compatible" content="IE=edge">
                            <meta name="author" content="BillionToOne">
                            <title>Clarity Work Log</title>
                            <link rel="icon" href="data:image/svg+xml;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iVVRGLTgiIHN0YW5kYWxvbmU9Im5vIj8+CjwhRE9DVFlQRSBzdmcgUFVCTElDICItLy9XM0MvL0RURCBTVkcgMS4xLy9FTiIgImh0dHA6Ly93d3cudzMub3JnL0dyYXBoaWNzL1NWRy8xLjEvRFREL3N2ZzExLmR0ZCI+Cjxzdmcgd2lkdGg9IjEwMCUiIGhlaWdodD0iMTAwJSIgdmlld0JveD0iMCAwIDE2NiAxNjciIHZlcnNpb249IjEuMSIgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIiB4bWxuczp4bGluaz0iaHR0cDovL3d3dy53My5vcmcvMTk5OS94bGluayIgeG1sOnNwYWNlPSJwcmVzZXJ2ZSIgeG1sbnM6c2VyaWY9Imh0dHA6Ly93d3cuc2VyaWYuY29tLyIgc3R5bGU9ImZpbGwtcnVsZTpldmVub2RkO2NsaXAtcnVsZTpldmVub2RkO3N0cm9rZS1saW5lam9pbjpyb3VuZDtzdHJva2UtbWl0ZXJsaW1pdDoyOyI+CiAgICA8ZyB0cmFuc2Zvcm09Im1hdHJpeCgxLDAsMCwxLC00NCwtOC44MjQ2KSI+CiAgICAgICAgPHBhdGggZD0iTTc1LjcyMSw2OC44MTJMMTA1Ljg4NiwxMTYuODcxTDQ0LDExNi41MjdMNzUuNzIxLDY4LjgxMloiIHN0eWxlPSJmaWxsOnJnYigyMzMsMjgsMTQzKTsiLz4KICAgIDwvZz4KICAgIDxnIHRyYW5zZm9ybT0ibWF0cml4KDEsMCwwLDEsLTQ0LC04LjgyNDYpIj4KICAgICAgICA8cGF0aCBkPSJNODUuMTU0LDU2Ljc2OUwxMTUuMjA1LDguODI1TDE0NS4xNDEsNTYuNDI1TDExNS4zMiwxMDMuNzk1TDg1LjE1NCw1Ni43NjlaIiBzdHlsZT0iZmlsbDpyZ2IoMjMzLDI4LDE0Myk7Ii8+CiAgICA8L2c+CiAgICA8ZyB0cmFuc2Zvcm09Im1hdHJpeCgxLDAsMCwxLC00NCwtOC44MjQ2KSI+CiAgICAgICAgPHBhdGggZD0iTTEyNy44MjIsMTE2LjgxM0wxNTQuNjYxLDY4LjE4MUwyMDkuODMyLDY4LjE4MUwxODQuMDI0LDExNi44MTNMMTI3LjgyMiwxMTYuODEzWiIgc3R5bGU9ImZpbGw6cmdiKDIzMywyOCwxNDMpOyIvPgogICAgPC9nPgogICAgPGcgdHJhbnNmb3JtPSJtYXRyaXgoMSwwLDAsMSwtNDQsLTguODI0NikiPgogICAgICAgIDxwYXRoIGQ9Ik0xMTMuMzcsMTI4LjI4M0wxNzguNzQ4LDEyOC4yODNMMTQ2LjE3NCwxNzUuNTM5TDExMy4zNywxMjguMjgzWiIgc3R5bGU9ImZpbGw6cmdiKDIzMywyOCwxNDMpOyIvPgogICAgPC9nPgo8L3N2Zz4K" type="image/svg+xml">
                        </head>
                        <body>
                        <style>
                        :root {
                           --med1: #42adeb; /* Blue */
                            --med2: #2ea537; /* Green */
                            --med3: #78878d; /* Bluegray */
                            --med4: #ad0000; /* Red */
                        
                            --light1: #D5F0FB; /* light blue */
                            --light2: #e8ffeb; /* light green */
                            --light3: #eef3f5; /* light bluegray */
                            --light4: #da6e6e; /* Dark Red */
                        
                            --dark1: #3383b1; /* Dark Blue */
                            --dark2: #3383b1; /* Dark Green */
                            --dark3: #475155; /* Dark Bluegray */
                            --dark4: #640000; /* Dark Red */
                        }
                        body {
                            margin:0px;
                            padding:0px;
                            font-family:Arial, sans;
                        }
                        td { border: solid 1px #adadad; padding: 4px 10px; }
                        table { border-collapse:collapse; background-color:#ffffff; margin-top:20px; }
                        thead { background-color:var(--dark1);color:#ffffff; }
                        .subhead { 
                            background-color: var(--light1); 
                            border-top: solid 2px var(--dark1);
                            font-weight: bold;
                            color: var(--dark1);
                        }
                        #header {
                            border-bottom:solid 1px #cccccc;
                            height:30px;
                            padding:5px;
                            line-height:30px;
                            background-color: var(--light1);
                        }
                        #header-img {
                            vertical-align: middle;
                            display: inline-block;
                            margin-right: 10px;
                            height:100%;
                        }
                        a.header-link {
                            display: inline-block;
                            line-height: 30px;
                            height: 30px;
                            vertical-align: middle;
                            font-family: sans-serif;
                            margin-left: 15px;
                            font-size: 18px;
                            font-weight: bold;
                            color: var(--med1);
                        }
                        a.header-link:hover {
                            color: var(--dark1);
                        }

                        .header-divider {
                            display:inline-block;
                            vertical-align: middle;
                            height:100%;
                            width:1px;
                            color:var(--dark1);
                        }

                        #subsite {
                            font-size: 24px;
                            color: #b1b1b1;
                            vertical-align: middle;
                        }
                        .log {
                            white-space: pre-wrap; /* css-3 */
                            white-space: -moz-pre-wrap !important; /* Mozilla, since 1999 */
                            white-space: -pre-wrap; /* Opera 4-6 */
                            white-space: -o-pre-wrap; /* Opera 7 */
                            word-wrap: break-word; /* Internet Explorer 5.5+ */
                            border: 1px double #cccccc;
                            padding: 20px;
                            background-color: #e9e9e9;
                            margin:10px;
                            margin-bottom: 100px;
                        }
                        .log_head {
                            font-size: 22px;
                            border: solid 2px var(--dark1);
                            padding: 20px;
                            background-color: #f3f3f3;
                            margin-bottom: 20px;
                            border-radius: 20px;
                            width: fit-content;
                        }
                        .script-name {
                            color: var(--med1);
                        }
                        .successful_automation {
                            background-color: #00a900;
                            margin: 10px;
                            padding: 10px;
                            width: fit-content;
                            color: white;
                            font-size: 20px;
                            border: solid 1px #004b00;
                            border-radius: 3px;
                            display:none;
                        }
                        .failed_automation {
                            background-color: #c10000;
                            margin: 10px;
                            padding: 10px;
                            width: fit-content;
                            color: white;
                            font-size: 20px;
                            border: solid 1px #550000;
                            border-radius: 3px;
                            display:none;
                        }
                        .successful_automation svg, .failed_automation svg {
                            height: 30px;
                            vertical-align: bottom;
                        }
                        .log_link {
                            color:var(--med1);
                        }
                        .log_link:hover {
                            color:var(--dark1);
                        }
                        .exception {
                            background-color: #fee9e9;
                            padding: 10px;
                            border: solid 2px #cc0000;
                            color: #740000;
                            white-space: pre-wrap; /* css-3 */
                            white-space: -moz-pre-wrap !important; /* Mozilla, since 1999 */
                            white-space: -pre-wrap; /* Opera 4-6 */
                            white-space: -o-pre-wrap; /* Opera 7 */
                            word-wrap: break-word; /* Internet Explorer 5.5+ */
                        }
                        .action-remove {
                            background-color: #ffdede;
                            border: solid 1px #cc0000;
                            padding: 2px 10px;
                            border-radius: 3px;
                        }
                        .action-nextstep {
                            background-color: #f4ffec;
                            border: solid 1px #00a841;
                            padding: 2px 10px;
                            border-radius: 3px;
                        }
                        .action-reflex {
                            background-color: #ecedff;
                            border: solid 1px #000ea8;
                            padding: 2px 10px;
                            border-radius: 3px;
                        }
                        .action-repeat {
                            background-color: #fdecff;
                            border: solid 1px #9600a8;
                            padding: 2px 10px;
                            border-radius: 3px;
                        }
                        eventtitle {
                            color:var(--dark1);
                            font-size:20px;
                            display:block;
                            margin-top:30px;
                            padding:5px;
                        }
                        eventdate {
                            padding: 5px 13px;
                            font-size:18px;
                        }
                        eventop {
                            padding: 5px 13px;
                            font-size:16px;
                            font-style:italic;
                        }
                        eventlog {
                            display:block;
                            background-color:var(--light3);
                            margin-left:50px;
                            padding:30px;
                        }
                        processchange,artifactchange,nochange,change {
                            display:block;
                        }
                        processchange {
                            background-color:var(--light2);
                        }
                        artifactchange {
                            background-color:var(--light1);
                        }
                        nochange {
                            background-color:var(--light3);
                        }
                        changeid {
                            display:inline;
                            margin-right:10px;
                            font-style:italic;
                            color:var(--med3);
                        }
                        caret {
                            display:block;
                            position:absolute;
                            height:20px;
                            vertical-align: bottom;
                            margin:0px 10px;
                            top:0px;
                            left:200px;
                            fill:var(--dark1);
                        }
                        caret svg {
                            height:100%;
                        }
                        epp caret {
                            display:inline-block;
                            position:static;
                        }
                        detail {
                            display:block;
                            margin-left: 150px;
                            position:relative;
                            height:25px;
                        }
                        udf {
                            display:block;
                            width:200px;
                            text-align:right;
                        }
                        udfvalue {
                            display:block;
                            position:absolute;
                            left:230px;
                            top:0px;
                        }
                        changeerror {
                            color: var(--light4);
                            margin-left: 150px;
                        }
                        changeinfo {
                            color: var(--med2);
                            margin-left: 150px;
                        }
                        details {
                            margin-top:20px;
                        }
                        eppname {
                            color:var(--med1);
                        }
                        summary {
                            color: var(--med3);
                        }
                        junk {
                            display:block;
                        }
                        container {
                            display:block;
                        }
                        .False {
                            background-color:#ff0000;
                        }
                        

                    </style>
                    <div id="header">
                        <div id="header-img">
                            <svg width="100%" height="100%" viewBox="0 0 554 130" version="1.1" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" xml:space="preserve" xmlns:serif="http://www.serif.com/" style="fill-rule:evenodd;clip-rule:evenodd;stroke-linejoin:round;stroke-miterlimit:2;">
                                <rect id="Artboard1" x="0" y="0" width="553.635" height="129.17" style="fill:none;"/>
                                <g id="Artboard11" serif:id="Artboard1">
                                    <g transform="matrix(1,0,0,1,-433.865,0)">
                                        <rect x="689.11" y="0" width="298.39" height="129.17" style="fill:rgb(28,172,233);"/>
                                    </g>
                                    <g transform="matrix(1.00353,0,0,1.0167,-23.0851,-0.2602)">
                                        <path d="M23.423,110.589L23.423,15.583L56.184,15.583C58.602,15.583 60.825,15.627 62.853,15.715C64.882,15.803 66.717,15.979 68.357,16.242C74.141,17.033 78.824,19.58 82.407,23.885C85.989,28.189 87.781,33.416 87.781,39.565C87.781,49.053 83.594,55.641 75.22,59.331C81.177,60.385 85.86,62.976 89.27,67.105C92.68,71.234 94.385,76.505 94.385,82.917C94.385,92.405 90.759,99.784 83.507,105.055C78.328,108.744 70.86,110.589 61.105,110.589L23.423,110.589ZM47.249,34.69L47.249,52.083L54.242,52.083C57.436,52.083 60.026,51.293 62.012,49.712C63.997,48.13 64.99,45.978 64.99,43.255C64.99,37.545 61.235,34.69 53.724,34.69L47.249,34.69ZM47.249,70.795L47.249,91.482L56.184,91.482C65.508,91.482 70.17,88.056 70.17,81.204C70.17,78.745 69.371,76.593 67.774,74.748C66.177,72.903 64.084,71.761 61.494,71.322L55.925,70.795L47.249,70.795Z" style="fill:rgb(48,59,74);fill-rule:nonzero;"/>
                                    </g>
                                    <g transform="matrix(1.00353,0,0,1.0167,-23.0851,-0.2602)">
                                        <path d="M100.33,37.193L100.33,15.583L165.465,15.583L165.465,37.193L144.876,37.193L144.876,110.589L121.049,110.589L121.049,37.193L100.33,37.193Z" style="fill:rgb(48,59,74);fill-rule:nonzero;"/>
                                    </g>
                                    <g transform="matrix(1.00353,0,0,1.0167,-23.0851,-0.2602)">
                                        <path d="M169.811,63.152C169.811,49.009 174.343,37.215 183.408,27.772C192.472,18.328 203.954,13.607 217.853,13.607C231.666,13.607 243.083,18.307 252.104,27.706C261.125,37.106 265.636,48.921 265.636,63.152C265.636,77.383 261.082,89.198 251.974,98.598C242.867,107.998 231.493,112.697 217.853,112.697C204.04,112.697 192.58,108.02 183.473,98.664C174.365,89.308 169.811,77.471 169.811,63.152ZM217.853,37.062C211.292,37.062 205.681,39.609 201.019,44.704C196.357,49.799 194.026,55.949 194.026,63.152C194.026,70.531 196.357,76.746 201.019,81.797C205.681,86.849 211.292,89.374 217.853,89.374C224.328,89.374 229.874,86.827 234.493,81.732C239.111,76.636 241.421,70.443 241.421,63.152C241.421,56.036 239.133,49.909 234.558,44.77C229.982,39.631 224.414,37.062 217.853,37.062Z" style="fill:rgb(48,59,74);fill-rule:nonzero;"/>
                                    </g>
                                    <g transform="matrix(0.87669,0,0,0.888191,261.462,9.08356)">
                                        <path d="M23.423,110.589L23.423,15.583L47.249,15.583L47.249,88.979L75.09,88.979L75.09,110.589L23.423,110.589Z" style="fill:white;fill-rule:nonzero;"/>
                                    </g>
                                    <g transform="matrix(0.87669,0,0,0.888191,261.462,9.08356)">
                                        <rect x="86.615" y="15.583" width="23.827" height="95.006" style="fill:white;"/>
                                    </g>
                                    <g transform="matrix(0.87669,0,0,0.888191,261.398,9.08356)">
                                        <path d="M123.65,110.589L135.046,15.583L158.095,15.583L174.8,63.547C175.577,66.007 177.549,72.118 177.549,72.118C177.549,72.118 179.721,65.743 180.498,63.547L197.202,15.583L220.252,15.583L231.518,110.589L208.339,110.589L204.066,67.369L202.718,59.231C202.718,59.231 201.735,61.439 199.922,66.71L184.642,110.589L170.527,110.589L155.247,66.71L152.972,59.624L151.88,67.369L146.83,110.589L123.65,110.589Z" style="fill:white;fill-rule:nonzero;"/>
                                    </g>
                                    <g transform="matrix(0.87669,0,0,0.888191,261.462,9.08356)">
                                        <path d="M241.521,98.466L249.938,79.36C257.621,88.408 265.434,92.932 273.376,92.932C276.743,92.932 279.592,92.01 281.923,90.165C284.254,88.32 285.419,85.904 285.419,82.917C285.419,80.633 284.383,78.701 282.311,77.12C280.757,76.065 276.786,74.44 270.398,72.244C262.628,69.609 257.19,66.973 254.082,64.338C248.211,59.331 245.276,52.742 245.276,44.573C245.276,35.876 248.535,28.563 255.053,22.633C261.571,16.703 269.75,13.739 279.592,13.739C290.124,13.739 299.404,17.252 307.433,24.28L297.98,41.805C291.505,36.008 285.505,33.109 279.98,33.109C277.218,33.109 274.887,33.877 272.988,35.415C271.088,36.952 270.139,38.906 270.139,41.278C270.139,43.562 271.002,45.451 272.729,46.944C273.937,48.086 278.016,49.777 284.966,52.018C291.915,54.258 297.073,56.827 300.44,59.726C306.224,64.558 309.116,71.322 309.116,80.019C309.116,89.506 305.879,97.324 299.404,103.473C292.93,109.623 284.513,112.697 274.153,112.697C261.204,112.697 250.326,107.954 241.521,98.466Z" style="fill:white;fill-rule:nonzero;"/>
                                    </g>
                                </g>
                            </svg>
                        </div>
                    <span id="subsite">Clarity Work Log</span></div>"""

caret = '<caret><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 256 512"><path d="M246.6 278.6c12.5-12.5 12.5-32.8 0-45.3l-128-128c-9.2-9.2-22.9-11.9-34.9-6.9s-19.8 16.6-19.8 29.6l0 256c0 12.9 7.8 24.6 19.8 29.6s25.7 2.2 34.9-6.9l128-128z"/></svg></caret>'

In [ ]:
# Define the timezone
timezone = pytz.timezone('America/Los_Angeles')  # Example: PST timezone

# Get the current time in the specified timezone
current_time = datetime.now(timezone)

# Format the date and time as 'YYYY-MM-DD HH:MM'
formatted_time = current_time.strftime('%Y-%m-%d %H:%M')
formatted_date = current_time.strftime('%Y-%m-%d')

file_name = fr'{dir1}\{formatted_date} routing log - {remove_from_stage}-{move_to_stage} ({len(artifact_ids)} samples)'

new_stage = None
old_stage = None

old_wf = lims.workflows.get_by_name(remove_from)
for stage in old_wf.stages:
    if stage.name==remove_from_stage:
        old_stage=stage
        print('From:',old_wf.name,'-',stage.name)
        break

if old_stage==None:
    raise Exception(f"Couldn't find {remove_from_stage} in workflow {old_wf.name}")

new_wf = lims.workflows.get_by_name(move_to)
for stage in new_wf.stages:
    if stage.name==move_to_stage:
        new_stage=stage
        print('To:',new_wf.name,'-',stage.name)
        break

if new_stage==None:
    raise Exception(f"Couldn't find {move_to_stage} in workflow {new_wf.name}")

In [ ]:
workflowuri=None
artifacts=[]
output_table={}
for a in artifact_ids: 
    artifact = Artifact(lims,uri=f'https://billiontoone-{server}.claritylims.com/api/v2/artifacts/{a}')

    if artifact.control_type=="True":
        print('ERROR: control type')
        break
        
    artifacts.append(artifact)
    # output_table[artifact.limsid]={'name':artifact.name,'LIMSID':artifact.limsid, 'current_status':'', 'current_wf':'','current_stage':'', 'verify_remove':False, 'verify_add':False}
    output_table[artifact.limsid]={'name':artifact.name,'LIMSID':artifact.limsid, 'current_status':'', 'verify_remove':False, 'verify_add':False}
    for x in artifact.workflow_stages:
        found_artifact = False
        if x.status=='IN_PROGRESS' or x.status=='QUEUED':
            if not x.stage.workflow.name == remove_from:
                continue
            workflowuri=x.uri
            output_table[artifact.limsid]['current_status']=x.status
            found_artifact=True
            break
    if found_artifact == False:
        raise Exception(f"Didn't find {artifact.name} in queue for {remove_from}")
    
    # print(artifact.name)
print('Processed',len(artifacts),'artifacts')
print('\nSamples Workflow:')
old_stage_uri=None
if workflowuri:
    old_stage=Stage(lims,uri=workflowuri)
    print(old_stage.workflow.name,'-',old_stage.name)
    print(old_stage.name)
    if not old_stage.workflow.name == remove_from:
        raise Exception('Wrong workflow')
else:
    print('NO WORKFLOW FOUND')

In [ ]:
if len(artifacts)>0 and workflowuri:
    print(f'Remove {len(artifacts)} artifacts')    
    router=Router(lims)
    router.unassign(old_stage.uri,artifacts)
    print(f'Reassign {len(artifacts)} artifacts') 
    router.assign(new_stage.uri,artifacts)
    router.commit()

In [ ]:
old_queue = lims.queues.from_limsid(old_stage.step.limsid)
old_queued_sampled = []
for q in old_queue.queued_artifacts:
    old_queued_sampled.append(q.artifact.limsid)
    # print(q.artifact.name)

new_queue = lims.queues.from_limsid(new_stage.step.limsid)
new_queued_sampled = []
for q in new_queue.queued_artifacts:
    new_queued_sampled.append(q.artifact.limsid)
    # print(q.artifact.name)

In [ ]:
import pdfkit
import sys
sys.path.append(r"C:\Program Files\wkhtmltopdf\bin")
options = {
    'print-media-type': '',
    'background': '',  # Enable background graphics
}

with open(f'{file_name}.html', "w+") as file:
    file.write(header)
    file.write(f'<br>{formatted_time}')
    file.write(f'<br><b>From:</b> {remove_from} [{old_wf.limsid}] - {remove_from_stage} [{old_stage.limsid}]')
    file.write(f'<br><b>To:</b> {move_to} [{new_wf.limsid}] - {move_to_stage} [{new_stage.limsid}]')
    file.write(f'<br><br><b>Samples Requested:</b> {len(artifact_ids)}')
    file.write('<table style="width:100%"><thead>')
    
    # Get the headers from the first item in the dictionary
    headers = [key for key in next(iter(output_table.values())).keys()]
    file.write(''.join(f'<td>{header if header.isupper() else header.replace("_", " ").capitalize()}</td>' for header in headers) + '</tr>')
    file.write('</thead>')
    # Write the rows
    for key, sample in output_table.items():
        sample['verify_remove'] = (not sample['LIMSID'] in old_queued_sampled)
        sample['verify_add'] = (sample['LIMSID'] in new_queued_sampled)
        file.write('<tr>')
        file.write(''.join(f'<td class="{value}">{value}</td>' for value in sample.values()))
        file.write('</tr>')
    file.write('</table>')
    file.write('<br><br>end of file.')

pdfkit.from_file(f'{file_name}.html', f'{file_name}.pdf', options=options)